In [1]:

import os
import numpy as np


FileNotFoundError: [Errno 2] No such file or directory: './data/mnist/2_semisupervised/train_x.npy'

In [26]:
# data_path = os.path.join('../data', 'mnist', '2_semisupervised')
data_path = os.path.join('../data', 'mnist', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
valid_x_data = np.load(os.path.join(data_path, 'valid_x.npy'))
test_x_data = np.load(os.path.join(data_path, 'test_x.npy'))
train_y_data = np.argmax(np.load(os.path.join(data_path, 'train_y.npy')), axis=1)
valid_y_data = np.argmax(np.load(os.path.join(data_path, 'valid_y.npy')), axis=1)

In [27]:
x_data = train_x_data
y_data = train_y_data

In [30]:
unlabeled_x = []
unlabeled_y = []
labeled_x = []
labeled_y = []
labaled_data = []

u, indices = np.unique(y_data, return_index=True)
print(u)
print(indices)
    

[0 1 2 3 4 5 6 7 8 9]
[13 29 37  7 22  0  1  3  8  9]


In [38]:
data_path = os.path.join('../data', 'mnist', '2_semisupervised')
# data_path = os.path.join('../data', 'mnist', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
valid_x_data = np.load(os.path.join(data_path, 'valid_x.npy'))
test_x_data = np.load(os.path.join(data_path, 'test_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))
valid_y_data = np.argmax(np.load(os.path.join(data_path, 'valid_y.npy')), axis=1)

In [39]:
x_data = train_x_data
y_data = train_y_data

In [47]:


exists = y_data.sum(axis=1)
u, counts = np.unique(exists, return_counts=True)
print(u)
print(counts)

[0. 1.]
[3200  800]


In [49]:
unlabeled_x = x_data[np.where(exists==0)]
unlabeled_y = y_data[np.where(exists==0)]
labeled_x = x_data[np.where(exists==1)]
labeled_y = y_data[np.where(exists==1)]


In [51]:
print(unlabeled_x.shape, unlabeled_y.shape, labeled_x.shape, labeled_y.shape)

(3200, 784) (3200, 10) (800, 784) (800, 10)


## Balance 함수 테스트

In [52]:
data_path = os.path.join('../data', 'mnist', '1_imbalanced')
# data_path = os.path.join('../data', 'mnist', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))


In [100]:
x_data = train_x_data
y_data = train_y_data

In [101]:
argmax_y_data = np.argmax(y_data, axis=1)
u, counts = np.unique(argmax_y_data, return_counts=True)
max_count = max(counts)
min_count = min(counts)

In [59]:
if max_count == min_count:
    return x_data, y_data

SyntaxError: 'return' outside function (<ipython-input-59-94e16f8332f7>, line 2)

In [102]:
num_of_data_for_fewer_classes = int((max_count / min_count)) - 1
print(num_of_data_for_fewer_classes)

99


In [103]:

for idx, count in zip(u, counts):
    if count != min_count:
        continue
    imbalanced_label_indexes = np.where(argmax_y_data == idx)
    x_data = np.vstack((x_data, np.tile(x_data[imbalanced_label_indexes], (num_of_data_for_fewer_classes, 1))))
    y_data = np.vstack((y_data, np.tile(y_data[imbalanced_label_indexes], (num_of_data_for_fewer_classes, 1))))
    

In [104]:
x_data.shape

(4000, 784)

In [105]:
y_data.shape

(4000, 10)

In [107]:
np.empty(1, dtype=int)

array([99])

In [108]:
np.int(1)

1

In [109]:
import torch
labels=np.array([1,2])
torch.nn.functional.one_hot(labels, 5)

TypeError: one_hot(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

## MNIST 하이퍼 파라메터 탐색

In [2]:
import sys
sys.path.append("../")
import os
import numpy as np

In [3]:
from models.resnet import resnet18, resnet34
from Answer import HomeworkDataset

wandb: Wandb version 0.8.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
cuda is available: 0


In [4]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_transforms = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
num_classes = 10


In [5]:
data_path = os.path.join('../data', 'mnist', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))
train_y_data = np.argmax(train_y_data, axis=1)
train_dataset = HomeworkDataset(train_x_data, train_y_data, 'mnist', test_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
ckpt_path='../output/mnist/2_semisupervised/191204-1423/best.pth'

In [57]:
from torch.autograd import Variable
import torch.nn.functional as F 
model = resnet18(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(ckpt_path))

pred_labels = np.array([], dtype=int)
confidences = np.array([], dtype=int)
for batch_data in train_loader:
    images, labels = batch_data 
    images = Variable(images).to(device)
    labels = Variable(labels).to(device)
    outputs = model(images)
    values, indices  = F.softmax(outputs, dim=None).max(axis=1)
    pred_labels = np.hstack([pred_labels, indices.cpu()])
    confidences = np.hstack([confidences, values.detach().cpu()])


/home/amazingguni/neural-network-homework/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [58]:
train_real_path = '/home/amazingguni/images/mnist/3_noisy/train_real.csv'
with open(train_real_path) as f:
    real_labels = [int(line[:-1]) for line in f.readlines()]
real_labels = np.array(real_labels)

In [61]:
total = pred_labels.shape[0]
correct = np.equal(pred_labels, real_labels).sum()
ratio = float(correct)/float(total) * 100
print(f'{correct}/{total}({ratio}%)')


confidence_indice = np.where(confidences > 0.8)
revised_pred_labels = pred_labels[confidence_indice]
revised_real_labels = real_labels[confidence_indice]
total = revised_pred_labels.shape[0]
correct = np.equal(revised_pred_labels, revised_real_labels).sum()
ratio = float(correct)/float(total) * 100
print(f'{correct}/{total}({ratio}%)')
# for pred, confidence, real in zip(pred_labels, pred_outputs_max, real_label):
#     if pred != real:
#         print(confidence)

3299/4000(82.475%)
2591/2754(94.08133623819899%)


## CIFAR 하이퍼 파라메터 탐색

In [69]:
import sys
sys.path.append("../")
import os
import numpy as np
from models.resnet import resnet18, resnet34
from Answer import HomeworkDataset

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
num_classes = 100

data_path = os.path.join('../data', 'cifar', '2_semisupervised')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))
train_y_data = np.argmax(train_y_data, axis=1)
train_dataset = HomeworkDataset(train_x_data, train_y_data, 'cifar', test_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
ckpt_path='../output/cifar/2_semisupervised/191204-1438/best.pth'

array([], shape=(1, 0), dtype=float64)

In [75]:
unlabeled_loader = train_loader
unlabeled_x_data = train_x_data

from torch.autograd import Variable
import torch.nn.functional as F 
model = resnet18(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(ckpt_path))

pred_labels = np.array([], dtype=int)
confidences = np.array([], dtype=int)
for batch_data in unlabeled_loader:
    images, labels = batch_data 
    images = Variable(images).to(device)
    outputs = model(images)
    values, indices  = F.softmax(outputs, dim=None).max(axis=1)
    pred_labels = np.hstack([pred_labels, indices.cpu()])
    confidences = np.hstack([confidences, values.detach().cpu()])

confidence_indice = np.where(confidences > 0.8)
print(confidence_indice[0].shape)

print(unlabeled_x_data[confidence_indice])
print(pred_labels[confidence_indice])
# train_x_data = np.concatenate((train_x_data, unlabeled_x_data), axis=0)
# train_y_data = np.concatenate((train_y_data, total_pred), axis=0)

/home/amazingguni/neural-network-homework/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


(23251,)
[[0.54901963 0.5568628  0.5411765  ... 0.48235294 0.5568628  0.6862745 ]
 [0.44705883 0.44705883 0.5176471  ... 0.05098039 0.12941177 0.12941177]
 [0.7647059  0.63529414 0.52156866 ... 0.25882354 0.17254902 0.1254902 ]
 ...
 [0.88235295 0.93333334 0.9372549  ... 0.28235295 0.2901961  0.28627452]
 [0.3764706  0.6156863  0.5137255  ... 0.5803922  0.4862745  0.6392157 ]
 [0.84313726 0.72156864 0.7137255  ... 0.8509804  0.8509804  0.85490197]]
[35 18 48 ... 25 34 91]


In [64]:


pred_labels = np.array([], dtype=int)
confidences = np.array([], dtype=int)
for batch_data in train_loader:
    images, labels = batch_data 
    images = Variable(images).to(device)
    labels = Variable(labels).to(device)
    outputs = model(images)
    values, indices  = F.softmax(outputs, dim=None).max(axis=1)
    pred_labels = np.hstack([pred_labels, indices.cpu()])
    confidences = np.hstack([confidences, values.detach().cpu()])


/home/amazingguni/neural-network-homework/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [65]:
train_real_path = '/home/amazingguni/images/cifar/3_noisy/train_real.csv'
with open(train_real_path) as f:
    real_labels = [int(line[:-1]) for line in f.readlines()]
real_labels = np.array(real_labels)

In [68]:
total = pred_labels.shape[0]
correct = np.equal(pred_labels, real_labels).sum()
ratio = float(correct)/float(total) * 100
print(f'{correct}/{total}({ratio}%)')



confidence_indice = np.where(confidences > 0.8)
revised_pred_labels = pred_labels[confidence_indice]
revised_real_labels = real_labels[confidence_indice]
total = revised_pred_labels.shape[0]
correct = np.equal(revised_pred_labels, revised_real_labels).sum()
ratio = float(correct)/float(total) * 100
print(f'{correct}/{total}({ratio}%)')
# for pred, confidence, real in zip(pred_labels, pred_outputs_max, real_label):
#     if pred != real:
#         print(confidence)

28530/40000(71.325%)
23473/25676(91.42000311575012%)


## Cifar Noisy 정정

In [177]:
import sys
sys.path.append("../")
import os
import numpy as np
from models.resnet import resnet18, resnet34
from Answer import HomeworkDataset
from torch.autograd import Variable
import torch.nn.functional as F 

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
num_classes = 100

data_path = os.path.join('../data', 'cifar', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))
train_y_data = np.argmax(train_y_data, axis=1)
train_dataset = HomeworkDataset(train_x_data, train_y_data, 'cifar', test_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
ckpt_path='../output/cifar/3_noisy/191205-1239/best.pth'

In [178]:
loader = train_loader
x_data = train_x_data
y_data = train_y_data

model = resnet18(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(ckpt_path))

pred_labels = np.array([], dtype=int)
confidences = np.array([], dtype=int)
for batch_data in loader:
    images, labels = batch_data 
    images = Variable(images).to(device)
    outputs = model(images)
    values, indices  = F.softmax(outputs, dim=None).max(axis=1)
    pred_labels = np.hstack([pred_labels, indices.cpu()])
    confidences = np.hstack([confidences, values.detach().cpu()])



# train_x_data = np.concatenate((train_x_data, unlabeled_x_data), axis=0)
# train_y_data = np.concatenate((train_y_data, total_pred), axis=0)

/home/amazingguni/neural-network-homework/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [179]:
train_real_path = '/home/amazingguni/images/cifar/3_noisy/train_real.csv'
with open(train_real_path) as f:
    real_labels = [int(line[:-1]) for line in f.readlines()]
real_labels = np.array(real_labels)

In [194]:
total = y_data.shape[0]
y_correct = np.equal(real_labels, y_data).sum()
print(f'origin correct: {y_correct}/{total}({float(y_correct)/float(total) * 100})')

revice_y_correct = np.equal(real_labels, pred_labels).sum()
print(f'pred correct: {revice_y_correct}/{total}({float(revice_y_correct)/float(total) * 100})')

pred_top_indice = np.where(confidences > 0.99)
pred_top = pred_labels[pred_top_indice]
pred_top_total = pred_top.shape[0]
pred_top_correct = np.equal(real_labels[pred_top_indice], pred_top).sum()
print(f'pred_top correct: {pred_top_correct}/{pred_top_total}({float(pred_top_correct)/float(pred_top_total) * 100})')

revice_y = np.where(confidences > 0.98, pred_labels, y_data)
revice_y_correct = np.equal(real_labels, revice_y).sum()
print(f'revice correct: {revice_y_correct}/{total}({float(revice_y_correct)/float(total) * 100})')

origin correct: 32067/40000(80.1675)
pred correct: 22908/40000(57.269999999999996)
pred_top correct: 9564/10062(95.05068574836017)
revice correct: 33600/40000(84.0)


## MNIST Noisy 정정

In [195]:
import sys
sys.path.append("../")
import os
import numpy as np
from models.resnet import resnet18, resnet34
from Answer import HomeworkDataset
from torch.autograd import Variable
import torch.nn.functional as F 

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_transforms = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
num_classes = 10

data_path = os.path.join('../data', 'mnist', '3_noisy')
train_x_data = np.load(os.path.join(data_path, 'train_x.npy'))
train_y_data = np.load(os.path.join(data_path, 'train_y.npy'))
train_y_data = np.argmax(train_y_data, axis=1)
train_dataset = HomeworkDataset(train_x_data, train_y_data, 'mnist', test_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
ckpt_path='../output/mnist/3_noisy/191205-1240/best.pth'

In [196]:
loader = train_loader
x_data = train_x_data
y_data = train_y_data

model = resnet18(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(ckpt_path))

pred_labels = np.array([], dtype=int)
confidences = np.array([], dtype=int)
for batch_data in loader:
    images, labels = batch_data 
    images = Variable(images).to(device)
    outputs = model(images)
    values, indices  = F.softmax(outputs, dim=None).max(axis=1)
    pred_labels = np.hstack([pred_labels, indices.cpu()])
    confidences = np.hstack([confidences, values.detach().cpu()])


/home/amazingguni/neural-network-homework/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [197]:
train_real_path = '/home/amazingguni/images/mnist/3_noisy/train_real.csv'
with open(train_real_path) as f:
    real_labels = [int(line[:-1]) for line in f.readlines()]
real_labels = np.array(real_labels)

In [200]:
total = y_data.shape[0]
y_correct = np.equal(real_labels, y_data).sum()
print(f'origin correct: {y_correct}/{total}({float(y_correct)/float(total) * 100})')

revice_y_correct = np.equal(real_labels, pred_labels).sum()
print(f'pred correct: {revice_y_correct}/{total}({float(revice_y_correct)/float(total) * 100})')

pred_top_indice = np.where(confidences > 0.95)
pred_top = pred_labels[pred_top_indice]
pred_top_total = pred_top.shape[0]
pred_top_correct = np.equal(real_labels[pred_top_indice], pred_top).sum()
print(f'pred_top correct: {pred_top_correct}/{pred_top_total}({float(pred_top_correct)/float(pred_top_total) * 100})')

revice_y = np.where(confidences > 0.98, pred_labels, y_data)
revice_y_correct = np.equal(real_labels, revice_y).sum()
print(f'revice correct: {revice_y_correct}/{total}({float(revice_y_correct)/float(total) * 100})')

origin correct: 3272/4000(81.8)
pred correct: 3418/4000(85.45)
pred_top correct: 2715/2854(95.12964260686756)
revice correct: 3672/4000(91.8)
